In [10]:
import os
import easyocr
import cv2
import re
reader = easyocr.Reader(["en"])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [34]:
def passport_no(str):
    patt = "\d{8}|d{9}|\w{2}\d{6}|\w{2}\d{7}|\w{3}\d{5}|\w{3}\d{6}"
    pass_no = re.findall(patt,str)
    
    return pass_no[0]

In [39]:
def extract_info(img_path):
    img = cv2.imread(img_path)
    all_text = []

    result = reader.readtext(img,paragraph=False)

    for (bbox,text,prob) in result:
        (t1,tr,br,b1) = bbox

        t1 = (int(t1[0]), int(t1[1]))
        tr = (int(tr[0]), int(tr[1]))
        br = (int(br[0]), int(br[0]))
        b1 = (int(b1[0]), int(b1[1]))
    
        #Remove non-ASCII characters to display clean text on the image
        text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
        all_text.append(text)
        
    all_info = {}
    
    extracted_text = (all_text)

    raw_text = ' '.join(extracted_text)
    
    
    patt = "\d{8}|d{9}|\w{2}\d{6}|\w{2}\d{7}|\w{3}\d{5}|\w{3}\d{6}"
    passport_no = re.findall(patt,raw_text)[0]
    
    passport_type = all_text[-2].replace("<", " ")[0]
    
    name_text = all_text[-2].split(passport_type)[1].replace("<"," ")
    total_name = name_text[4:]

    first_name = total_name.split(" ")[0]
    given_name = total_name.split(first_name)[1]
    given_name = given_name.rstrip()
    given_name = given_name.lstrip()
    
    date_pat = "[0-9]{2} \w{3} \d{4}|\d{2} \w{3} \d{3} \d{1}"
    all_date =  re.findall(date_pat,raw_text)
    
    birth_date = all_date[0]
    issue_date = all_date[1]
    expiry_date = all_date[-1]
    
    nat = raw_text.split(given_name)[1]
    nationality = nat.split(birth_date)[0]
    nationality = nationality.rstrip()
    nationality = nationality.lstrip()
    
    place = raw_text.split(birth_date)
    birth_place = place[1].split(issue_date)[0]
    birth_place = birth_place.lstrip()
    birth_place = birth_place.rstrip()
    
    
    
    all_info["Passport No"] = passport_no
    all_info["passport type"] = passport_type
    all_info["Surname"] = first_name
    all_info["Given Name"] = given_name
    all_info["Date Of Birth"] = birth_date
    all_info["Date Of Isuue"] = issue_date
    all_info["Date Of Expiry"] = expiry_date
    all_info["Place Of Birth"] = birth_place
    all_info["Nationality"] = nationality
    
    return all_info


In [40]:
extract_info("nzpassport.png")

{'Passport No': 'LDQ01148',
 'passport type': 'P',
 'Surname': 'WhAKAATURANGA',
 'Given Name': 'FRED WIREMU John',
 'Date Of Birth': '28 FEB 1964',
 'Date Of Isuue': '05 oct 2009',
 'Date Of Expiry': '05 oct 201 4',
 'Place Of Birth': 'TAIHAPE',
 'Nationality': 'NEW ZEALAND Fan -'}